In [26]:
from tensor import TensorSpec, Tensor, TensorShape
from utils.index import Index
from collections.dict import Dict, KeyElement
from random import rand

In [27]:
@value
struct IntKey(KeyElement):
    var n: Int

    fn __init__(inout self, owned n: Int):
        self.n = n

    fn __hash__(self) -> Int:
        return hash(self.n)

    fn __eq__(self, other: Self) -> Bool:
        return self.n == other.n


let fashion_mnist_key = Dict[IntKey, String]()
fashion_mnist_key[0] = "T-shirt/top"
fashion_mnist_key[1] = "Trouser"
fashion_mnist_key[2] = "Pullover"
fashion_mnist_key[3] = "Dress"
fashion_mnist_key[4] = "Coat"
fashion_mnist_key[5] = "Sandal"
fashion_mnist_key[6] = "Shirt"
fashion_mnist_key[7] = "Sneaker"
fashion_mnist_key[8] = "Bag"
fashion_mnist_key[9] = "Ankle boot"


In [28]:
fn uint8_to_int32(uint8_tensor: Tensor[DType.uint8]) raises -> SIMD[DType.int32, 1]:
    let result = (
        (uint8_tensor[0].to_int() << 24)
        | (uint8_tensor[1].to_int() << 16)
        | (uint8_tensor[2].to_int() << 8)
        | uint8_tensor[3].to_int()
    )
    return SIMD[DType.int32, 1](result)


In [29]:
fn get_slice[type: DType](tensor: Tensor[type], start_index: Int, end_index: Int) raises -> Tensor[type]:
    if end_index < start_index:
        raise "End index more than start index."
    elif end_index == start_index:
        var output_tensor = Tensor[type](1)
        output_tensor[0] = tensor[start_index]

        return output_tensor
    else:
        var output_tensor = Tensor[type](end_index - start_index)
        for i in range(start_index, end_index):
            output_tensor[i - start_index] = tensor[i]
        return output_tensor


In [30]:
fn read_data_as_images(images_path: Path) raises -> Tensor[DType.uint8]:
    if images_path.exists():
        let image_file = images_path.read_bytes().astype[DType.uint8]()
        let num_images = uint8_to_int32(get_slice[DType.uint8](image_file, 4, 8))
        let width = uint8_to_int32(get_slice[DType.uint8](image_file, 8, 12))
        let height = uint8_to_int32(get_slice[DType.uint8](image_file, 12, 16))

        var image_data = get_slice(image_file, 16, image_file.num_elements())

        let images_shape = TensorShape(
            num_images.to_int(), height.to_int(), width.to_int()
        )

        let images = image_data.reshape(images_shape)
        return images
    raise "The images directory does not exist."


In [31]:
fn read_data_as_labels(labels_path: Path) raises -> Tensor[DType.uint8]:
    if labels_path.exists():
        let label_file = labels_path.read_bytes().astype[DType.uint8]()
        let num_labels = uint8_to_int32(get_slice[DType.uint8](label_file, 4, 8))

        let labels = get_slice(label_file, 8, num_labels.to_int() + 8)

        return labels
    raise "The labels directory does not exist."


In [32]:
fn tensor_print[type: DType](index: Int, tensor: Tensor[type]):
    var cur_line: String

    fn get_char_for_pixel(pixel_value: Int) -> String:
        if pixel_value == 0:
            return " "
        elif pixel_value < 32:
            return "."
        elif pixel_value < 64:
            return ","
        elif pixel_value < 96:
            return ":"
        elif pixel_value < 128:
            return ";"
        elif pixel_value < 160:
            return "o"
        elif pixel_value < 192:
            return "O"
        elif pixel_value < 224:
            return "X"
        else:
            return "#"

    for j in range(tensor.shape()[1]):
        cur_line = ""
        for k in range(tensor.shape()[2]):
            cur_line += get_char_for_pixel(tensor[Index(index, j, k)].to_int()) + " "
        print(cur_line)


In [34]:
let base_dir = "/Users/tprazak/Documents/seminary_work_nn/MNIST_in_mojo/"
let image_path_mnist = base_dir + "mnist/train-images.idx3-ubyte"
let labels_path_mnist = base_dir + "mnist/train-labels.idx1-ubyte"
let image_path_fashion = base_dir + "fashion_mnist/train-images-idx3-ubyte"
let labels_path_fashion = base_dir + "fashion_mnist/train-labels-idx1-ubyte"
let images = read_data_as_images(image_path_mnist)
let labels = read_data_as_labels(labels_path_mnist)
let fashion_images = read_data_as_images(image_path_fashion)
let fashion_labels = read_data_as_labels(labels_path_fashion)


In [35]:
struct DenseLayer:
    var weights: Tensor[DType.int16]
    var biases: Tensor[DType.int16]

    fn __init__(inout self, num_inputs: Int, num_neurons: Int):
        self.weights = Tensor[DType.int16]()
        self.biases = Tensor[DType.int16]()
        

In [36]:
print(labels[911])


7
